In [3]:
import pandas as pd
from pathlib import Path
import xarray as xr
import numpy as np
import calendar
import os.path

In [4]:
zarr_ds = xr.open_zarr(store='/home/jovyan/shared/data/INDIAN_OCEAN_025GRID_DAILY.zarr', consolidated=True)
zarr_ds

<xarray.Dataset>
Dimensions:          (time: 16071, lat: 177, lon: 241)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -11.5 -11.75 -12.0
  * lon              (lon) float32 42.0 42.25 42.5 42.75 ... 101.5 101.8 102.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/19)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    curr_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    curr_speed       (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    ...               ...
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    wind_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
    wind_speed       (time, lat, lon) float32 dask.array<chunksize=(100, 177, 241), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-19
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [7]:
import pickle
with open('/home/jovyan/shared/jiarui-notebooks/sampled_points_50km.pickle', 'rb') as handle:
    sampled_points_50km = pickle.load(handle)

with open('/home/jovyan/shared/jiarui-notebooks/nearest_points.pickle', 'rb') as handle1:
    nearest_points = pickle.load(handle1)

In [12]:
# Define latitude and longitude range as slices
lats = slice(35, -5)
lons = slice(45, 90)

# This function finds the nearest sea surface temperature (SST) for a given point using a spatial tree.
def find_nearest_sst(point, tree, ds):
    # Get coordinates from the point
    lon, lat = point.x, point.y
    # Query the tree to find the nearest point and get its index
    _, idx = tree.query(np.array([lon, lat]), k=1)
    # Use the index to get the corresponding SST from the data
    nearest_sst = ds['sst'].values.ravel()[idx]
    # Return the SST
    return nearest_sst

# This function calculates the difference in SST between two sets of points
def get_sst_difference(ds, points1, points2, tree):
    # Initialize an empty list to hold the SST differences
    sst_diff = []
    # Iterate over pairs of points
    for point1, point2 in zip(points1, points2):
        # Find the SST for each point
        sst1 = find_nearest_sst(point1, tree, ds)
        sst2 = find_nearest_sst(point2, tree, ds)
        # Calculate the difference and append it to the list
        sst_diff.append(sst1 - sst2)
    return sst_diff

In [8]:
zarr_new = zarr_ds.sel(lat=slice(35, -5), lon=slice(45,90))

all_nan_dates = np.isnan(zarr_new["sst"]).all(dim=["lon", "lat"]).compute()

zarr_ds = zarr_new.sel(time=all_nan_dates == False)

zarr_ds

<xarray.Dataset>
Dimensions:          (time: 15058, lat: 149, lon: 181)
Coordinates:
  * lat              (lat) float32 32.0 31.75 31.5 31.25 ... -4.5 -4.75 -5.0
  * lon              (lon) float32 45.0 45.25 45.5 45.75 ... 89.5 89.75 90.0
  * time             (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2022-12-31
Data variables: (12/19)
    CHL              (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    CHL_uncertainty  (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    adt              (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    air_temp         (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    curr_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    curr_speed       (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    ...               ...
    ug_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    v_curr           (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    v_wind           (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    vg_curr          (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    wind_dir         (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
    wind_speed       (time, lat, lon) float32 dask.array<chunksize=(100, 149, 181), meta=np.ndarray>
Attributes: (12/17)
    creator_email:              minhphan@uw.edu
    creator_name:               Minh Phan
    creator_type:               person
    date_created:               2023-07-19
    geospatial_lat_max:         32.0
    geospatial_lat_min:         -12.0
    ...                         ...
    geospatial_lon_units:       degrees_east
    source:                     Earth & Space Research (ESR), Copernicus Clim...
    summary:                    Daily mean of 0.25 x 0.25 degrees gridded dat...
    time_coverage_end:          2022-12-31T23:59:59
    time_coverage_start:        1979-01-01T00:00:00
    title:                      Climate Data for Coastal Upwelling Machine Le...

In [24]:
import dask.array as da

def get_nearest_idx(dataset, lat, lon):
    lat_idx = abs(dataset.lat - lat).argmin().item()
    lon_idx = abs(dataset.lon - lon).argmin().item()
    return (lat_idx, lon_idx)

# Precompute indices for all pairs of points
near_indices = [get_nearest_idx(zarr_ds["sst"], pt.y, pt.x) for pt in sampled_points_50km.geometry]
off_indices = [get_nearest_idx(zarr_ds["sst"], pt.y, pt.x) for pt in nearest_points.geometry]

# Convert indices to dask arrays for vectorized operations
near_lat_indices, near_lon_indices = zip(*near_indices)
near_lat_indices = da.from_array(near_lat_indices)
near_lon_indices = da.from_array(near_lon_indices)

off_lat_indices, off_lon_indices = zip(*off_indices)
off_lat_indices = da.from_array(off_lat_indices)
off_lon_indices = da.from_array(off_lon_indices)

data_list = []

# Iterate over times in the dataset
for time in zarr_ds["time"].values:
    print(f"Processing for time: {time}")
    subset = zarr_ds.sel(time=time)
    
    # Vectorized SST extraction
    sst_near_values = subset["sst"].isel(lat=near_lat_indices, lon=near_lon_indices).values
    sst_off_values = subset["sst"].isel(lat=off_lat_indices, lon=off_lon_indices).values

    # Filter out nan values using dask
    valid_mask = ~da.logical_or(da.isnan(sst_near_values), da.isnan(sst_off_values))
    sst_near_values, sst_off_values = da.compute(sst_near_values[valid_mask], sst_off_values[valid_mask])
    
    # Calculate differences and store results
    sst_diff_vals = sst_near_values - sst_off_values
    for near_val, off_val, diff_val, near_point, off_point in zip(sst_near_values, sst_off_values, sst_diff_vals, sampled_points_50km.geometry, nearest_points.geometry):
        data_list.append([time, near_point, off_point, near_val, off_val, diff_val])

# Convert data list to DataFrame and save to CSV
df = pd.DataFrame(data_list, columns=["time", "near_shore_point", "off_shore_point", "sst_near_shore", "sst_off_shore", "sst_diff"])
df.to_csv('/home/jovyan/shared/jiarui-notebooks/sst_difference.csv', index=False)

Processing for time: 1979-01-01T00:00:00.000000000
Processing for time: 1979-01-02T00:00:00.000000000
Processing for time: 1979-01-03T00:00:00.000000000
Processing for time: 1979-01-04T00:00:00.000000000
Processing for time: 1979-01-05T00:00:00.000000000
Processing for time: 1979-01-06T00:00:00.000000000
Processing for time: 1979-01-07T00:00:00.000000000
Processing for time: 1979-01-08T00:00:00.000000000
Processing for time: 1979-01-09T00:00:00.000000000
Processing for time: 1979-01-10T00:00:00.000000000
Processing for time: 1979-01-11T00:00:00.000000000
Processing for time: 1979-01-12T00:00:00.000000000
Processing for time: 1979-01-13T00:00:00.000000000
Processing for time: 1979-01-14T00:00:00.000000000
Processing for time: 1979-01-15T00:00:00.000000000
Processing for time: 1979-01-16T00:00:00.000000000
Processing for time: 1979-01-17T00:00:00.000000000
Processing for time: 1979-01-18T00:00:00.000000000
Processing for time: 1979-01-19T00:00:00.000000000
Processing for time: 1979-01-20

In [26]:
df.iloc[:50]

,time,near_shore_point,off_shore_point,sst_near_shore,sst_off_shore,sst_diff
0,1979-01-01,POINT (51.326946552910215 10.294813025307628),POINT (53.18534593242639 9.306805982572545),299.000000,299.500000,-0.500000
1,1979-01-01,POINT (50.663989599968566 8.412538739972765),POINT (52.56440774778988 7.5580774155731),301.119781,300.927094,0.192688
2,1979-01-01,POINT (49.80726365696406 6.600737914177559),POINT (51.61212074557685 5.608759697278907),301.000000,300.994781,0.005219
3,1979-01-01,POINT (48.76233593503475 4.892051993872839),POINT (50.3633818641899 3.6323204285740767),300.812500,300.932281,-0.119781
4,1979-01-01,POINT (47.50386440681609 3.3328843382296682),POINT (49.005019289066624 1.9928376966654757),300.937500,300.932281,0.005219
5,1979-01-01,POINT (46.067412920452135 1.9460190134547457),POINT (47.290436573533825 0.3834632131107477),300.937500,301.119781,-0.182281
6,1979-01-01,POINT (89.9528980740787 21.48696381945197),POINT (89.99071000942922 19.241976375378698),300.750000,301.125000,-0.375000
7,1979-01-01,POINT (88.0916096060536 21.235197238795525),POINT (89.20214823709537 19.070595131736923),300.192719,301.062500,-0.869781
8,1979-01-01,POINT (86.93804114434356 19.954283204877978),POINT (88.54947013055305 18.353995137641196),300.192719,301.062500,-0.869781
9,1979-01-01,POINT (85.21337348804884 18.975717755058128),POINT (86.76709250388156 17.3094969923187),299.755219,301.000000,-1.244781
